### Actividad: Análisis "La Mañanera"
<p><kbd>0226594</kbd> Sara Carolina Gómez Delgado  </p>
Introducción a Python y análisis del lenguaje natural.

<kbd>https://www.youtube.com/watch?v=FILPdMMnYuA&ab_channel=Adri%C3%A1n</kbd>


In [6]:
from bs4 import BeautifulSoup
from datetime import datetime
import urllib.request
import glob
import re
import io
import os
import requests

## Read HTML code form URL

In [7]:
def getHTML(url, path):
    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'})
    with open(path, "w") as f:
        f.write(response.text)
getHTML("https://presidente.gob.mx/secciones/version-estenografica/page/2/", "../data/Política/html-code.txt")

In [8]:
regex = '(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?'
re.findall(regex, open("../data/Política/html-code.txt", "r", encoding = "utf-8").read())[:3]

[('http',
  '151.394.mwp.accessdomain.com',
  '/wp-content/themes/diplomat/favicon.ico'),
 ('https', 'presidente.gob.mx', '/feed/'),
 ('https', 'presidente.gob.mx', '/xmlrpc.php')]

## Fetch each page

In [9]:
url_pagina_i = "https://presidente.gob.mx/secciones/version-estenografica/page/"
num_paginas = 69
for i in range(num_paginas): 
    getHTML(url_pagina_i + str(i + 1) + "/", "./paginas/" + str(i + 1) + ".txt")

In [ ]:
urls_conf = []
for f_pagina in glob.glob("./paginas/*"):
    urls_conf += [path for _, _, path in re.findall(regex, open(f_pagina, "r", encoding = "utf-8").read()) if "estenografica-de-la" in path]